In [29]:
import pandas as pd
import numpy as np
import json
import mysql.connector
import datetime
import pytz

In [30]:
with open('issuer_to_masterid.json', 'r') as f:
    isinToMasterid = json.load(f)
with open('col_to_indicator_map.json', 'r', encoding='utf-8') as f:
    colToIndicator = json.load(f)
map_df = pd.read_csv('new_issuer_to_masterid.csv',sep=';')

In [31]:
today_date = '2022-01-11'
path = r'D:\\sriram\\agrud\\cbonds_data_entry\\server_files\\zipfiles\\data_files\\'+today_date+'\\'
try:
    df = pd.read_excel(path+'issuers.xls',dtype={'CIK number':str})
except:
    df = pd.read_csv(path+'issuers.csv', sep=';', encoding='latin')
df.columns = df.columns.str.strip().str.lower()

In [32]:
df.loc[df['full issuer name (eng)'].isna(), 'full issuer name (eng)'] = df.loc[df['full issuer name (eng)'].isna(), 'issuer short name (eng)']

In [33]:
new_df = pd.DataFrame()
for dt in map_df.to_dict(orient='records'):
    for i,row in df.iterrows():
        if dt['Full issuer name (eng)'] == row['full issuer name (eng)']:
            row['master id'] = dt['master id']
            new_df = new_df.append(row)
            break

In [34]:
# df['master id'] = df['full issuer name (eng)'].map(isinToMasterid)
new_df = new_df[new_df['master id'].notna()]
binaryColumns = ['company doesn`t exist (yes/no)','spv (yes/no)','international issuer (yes/no)']
new_df[binaryColumns] = new_df[binaryColumns].replace([[0], [1]], [["No"], ["Yes"]])

In [35]:
ts_date = '2020-01-01'
result = []
for i, row in new_df.iterrows():
    row2 = row.to_dict()
    masterId = row2['master id']
    for k, v in row2.items():
        if k in colToIndicator and pd.isna(v) == False:
            indicatorId = colToIndicator[k]
            if type(v) == str:
                json_data = json.dumps({'TEXT':v})
                dataType = 3
                value_data = 0 
            result.append([masterId,indicatorId,value_data,json_data,dataType,ts_date])

In [36]:
# df2 = pd.DataFrame(result)
# cols=['masterid','indicatorid','value_data','json_data','datatype','ts_date']
# df2.columns = cols
# df2.to_csv('issuers_data.csv',index=False,columns=cols)

In [37]:
# insert query
try:
        db_conn = mysql.connector.connect(host='54.237.79.6',user='rentech_user',database = 'rentech_db',password='N)baegbgqeiheqfi3e9314jnEkekjb',auth_plugin='mysql_native_password')
        cursor = db_conn.cursor()
        sql = """INSERT INTO `raw_data` (`id`, `master_id`, `indicator_id`, `value_data`, `json_data`, `data_type`, `ts_date`, `ts_hour`, `job_id`, `timestamp`) VALUES 
        (NULL, %s, %s, %s, %s, %s, %s, '0:0:0', 9, NOW()) ON DUPLICATE KEY UPDATE  
        master_id = VALUES(master_id), indicator_id = VALUES(indicator_id), value_data = VALUES(value_data), json_data = VALUES(json_data),data_type = VALUES(data_type), ts_date = VALUES(ts_date) ,ts_hour = VALUES(ts_hour),
        job_id = VALUES(job_id), batch_id = VALUES(batch_id);"""
        cursor.executemany(sql, result)
        print (f"{cursor.rowcount} rows are inserted into db")
        db_conn.commit()
except Exception as e:
        print ("Mysql Error:",e)
finally:
        if(db_conn.is_connected()):
                cursor.close()
                db_conn.close()
                print("MySQL connection is closed")

28 rows are inserted into db
MySQL connection is closed
